In [41]:
# this is the library that we will use to create break times in order to mimic human behaviour 
import time
import requests
from bs4 import BeautifulSoup
import re
from itertools import islice

# this library will be used for getting your credentials but keeping them hidden in the script 
from getpass import getpass

# Juicy stuff- these are the Classes we will use for interaction with a webpage:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# library for interacting with the operating system
import os

# you know pandas it's your best buddy 
import pandas as pd

# library for directory location:

import pathlib
import os
from os.path import join

#Ignore warning -- Some methods are going to be deprecated and I didn't change all (mainly in the function scrapper)
import warnings
warnings.filterwarnings('ignore')

In [42]:
#where are we going?
place = input('What location do you want to search for: ')

What location do you want to search for: Barcelona


### Booking.com for flight information

In [43]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [44]:
#Booking.com
#Go to the webpage
driver.get('https://www.booking.com')
time.sleep(5)

#accept cookies
accept = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept.click()
time.sleep(5)

#Go to flights menu
search = driver.find_element(By.LINK_TEXT, 'Flights')
search.click()
time.sleep(5)

#Where to?
search = driver.find_element(By.CSS_SELECTOR, 'input.css-g0pg3f-SearchboxInput')
search.click()
time.sleep(3)
search = driver.find_element(By.CSS_SELECTOR, 'input.css-1tl2oa1')
time.sleep(2)
search.send_keys(place)
search.send_keys(Keys.ENTER)
search.send_keys(Keys.ENTER)
time.sleep(2)
search = driver.find_element(By.CSS_SELECTOR,'div.css-1lc7njo')
search.click()



In [45]:
#press search button
search = driver.find_element(By.CSS_SELECTOR,'button.css-ya5gr9.wide')
search.send_keys(Keys.ENTER)
time.sleep(5)

In [46]:
#Flights page
time.sleep(10)
flights_page = driver.page_source
time.sleep(5)
soup = BeautifulSoup(flights_page, 'lxml')
soup = soup.find_all('div', attrs={'class':'css-17f6u3r-searchResultsList'})
flights = soup[0]


In [47]:
len(soup)

1

In [48]:
#Information for the first recommended flight
first_flight = flights.find_all('div', attrs={'id':'flightcard-0'})[0]

#Other informations we need 
price = first_flight.find_all('div', attrs={'class':'css-vxcmzt'})[0].text
Going_flight_departure_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_departure_time'})[0].text
Returning_flight_departure_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_departure_time'})[1].text
Going_flight_arrival_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_destination_time'})[0].text
returning_flight_arrival_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_destination_time'})[1].text
Going_flight_duration = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_duration'})[0].text
Returning_flight_duration = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_duration'})[1].text
Flight_carriers_going = first_flight.find_all('div', attrs={'data-testid':'flight_card_carriers_carrier'})[0].text
Flight_carriers_returning = first_flight.find_all('div', attrs={'data-testid':'flight_card_carriers_carrier'})[1].text

First_flight = pd.DataFrame({'Carrier' : Flight_carriers_going, 
                            "Price" : price, 
                            "Going Flight (departure and arrival)": (Going_flight_departure_time + " ------ " + Going_flight_duration + "------"+ Going_flight_arrival_time),
                            "Returning Flight (departure and arrival)": (Returning_flight_departure_time +" ------ " + Returning_flight_duration + "------"+ returning_flight_arrival_time)}, 
                            index=[0])
First_flight


,Carrier,Price,Going Flight (departure and arrival),Returning Flight (departure and arrival)
0,Iberia,"86,61 €",07:00 ------ 1h 15m------08:15,14:30 ------ 1h 25m------15:55


In [49]:
#The dataframe for all the recommended flights sorted by lowest price
main_flights_info = pd.DataFrame(columns= ["Carrier", 'Price', 'Going Flight (departure and arrival)', 'Returning Flight (departure and arrival)'])

for i in range(15):
    first_flight = flights.find_all('div', attrs={'id':'flightcard-'+str(i)})[0]

    price = first_flight.find_all('div', attrs={'class':'css-vxcmzt'})[0].text
    Going_flight_departure_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_departure_time'})[0].text
    Returning_flight_departure_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_departure_time'})[1].text
    Going_flight_arrival_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_destination_time'})[0].text
    returning_flight_arrival_time = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_destination_time'})[1].text
    Going_flight_duration = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_duration'})[0].text
    Returning_flight_duration = first_flight.find_all('div', attrs={'data-testid':'flight_card_segment_duration'})[1].text
    Flight_carriers_going = first_flight.find_all('div', attrs={'data-testid':'flight_card_carriers_carrier'})[0].text
    Flight_carriers_returning = first_flight.find_all('div', attrs={'data-testid':'flight_card_carriers_carrier'})[1].text

    First_flight = pd.DataFrame({'Carrier' : Flight_carriers_going, 
                                "Price" : price, 
                                "Going Flight (departure and arrival)": (Going_flight_departure_time + " ------ " + Going_flight_duration + "------"+ Going_flight_arrival_time),
                                "Returning Flight (departure and arrival)": (Returning_flight_departure_time +" ------ " + Returning_flight_duration + "------"+ returning_flight_arrival_time)}, 
                                index=[0])
    main_flights_info = pd.concat([main_flights_info,First_flight])
    main_flights_info = main_flights_info.sort_values('Price', ascending=True).reset_index(drop=True)

In [50]:
main_flights_info

,Carrier,Price,Going Flight (departure and arrival),Returning Flight (departure and arrival)
0,Air Europa,"100,65 €",07:30 ------ 1h 25m------08:55,14:30 ------ 1h 25m------15:55
1,Iberia,"101,62 €",07:00 ------ 1h 15m------08:15,18:05 ------ 1h 25m------19:30
2,Iberia,"103,62 €",09:00 ------ 1h 15m------10:15,13:30 ------ 1h 25m------14:55
3,Iberia,"103,62 €",20:00 ------ 1h 20m------21:20,13:30 ------ 1h 25m------14:55
4,Iberia,"104,62 €",12:25 ------ 1h 20m------13:45,14:30 ------ 1h 25m------15:55
5,Air Europa,"106,72 €",07:30 ------ 1h 25m------08:55,13:30 ------ 1h 25m------14:55
6,Iberia,"110,62 €",09:00 ------ 1h 15m------10:15,12:35 ------ 1h 25m------14:00
7,Iberia,"110,62 €",12:25 ------ 1h 20m------13:45,13:30 ------ 1h 25m------14:55
8,Iberia,"112,62 €",09:00 ------ 1h 15m------10:15,18:05 ------ 1h 25m------19:30
9,Iberia,"112,62 €",19:00 ------ 1h 15m------20:15,14:30 ------ 1h 25m------15:55


### Tripadvisor for restaurants, to-dos and tours

In [51]:
#Load the driver into the website and search for the place
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.tripadvisor.com/')
time.sleep(5)
accept = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
accept.click()
time.sleep(3)
search = driver.find_element(By.XPATH, '//*[@id="lithium-root"]/main/div[3]/div/div/div/form/input[1]')
search.click()
time.sleep(2)
search.send_keys(place)
search.send_keys(Keys.ENTER)

In [52]:
#RESTAURANTS
time.sleep(5)
restaurants = driver.find_element(By.XPATH, '//*[@id="search-filters"]/ul/li[4]/a')
restaurants.click()
time.sleep(5)

resaurants_page = driver.page_source
time.sleep(5)
soup2 = BeautifulSoup(resaurants_page, 'lxml')
restaurant_names = soup2.find_all('div', attrs={'class':'result-title'})
addresses_names = soup2.find_all('div',attrs={'class':'address-text'})

#Get the lists of information
restaurant_list = [restaurant.text for restaurant in restaurant_names]
addresses_list = [adress.text for adress in addresses_names]
restaurant_addresses = {restaurant_list[i]: addresses_list[i] for i in range(len(restaurant_list))}


In [53]:
def take(n, iterable):
    return list(islice(iterable, n))

#Recommend 5 restaurants with it's address
restaurant_recommendations = take(5, restaurant_addresses.items())
restaurant_recommendations_df = pd.DataFrame(restaurant_recommendations, columns=("Restaurant Name","Adress"))
restaurant_recommendations_df

,Restaurant Name,Adress
0,Bodega Biarritz 1881 Tapas bar,"Carrer Nou de Sant Francesc, 7, Barcelona, Cat..."
1,Barcelona Tapas Puerto Vallarta,"Calle Matamoros 906, Puerto Vallarta, Mexico"
2,Colom,"Escudellers, 33, Barcelona, Catalonia, Spain"
3,Le Romane,"Carrer Muntaner 103, Barcelona, Catalonia, Spain"
4,La Patsa Lab,"Calle Casanova 94, Barcelona, Catalonia, Spain"


In [54]:
### To Do's
to_dos = driver.find_element(By.XPATH, '//*[@id="search-filters"]/ul/li[5]/a')
to_dos.click()
time.sleep(5)

to_dos_page = driver.page_source
time.sleep(5)
soup3 = BeautifulSoup(to_dos_page, 'lxml')
to_dos = soup3.find_all('div', attrs={'class':'result-title'})

#Recommend 3 things to do in Barcelona
to_dos_list = [todo.text for todo in to_dos]
to_dos_list[:3]


['Barcelona Cathedral', 'Basilica de la Sagrada Familia', 'Parc Guell']

In [55]:
activities = pd.DataFrame(to_dos_list)
activities

,0
0,Barcelona Cathedral
1,Basilica de la Sagrada Familia
2,Parc Guell
3,Gothic Quarter (Barri Gotic)
4,Barcelona Day Tours
5,Free Walking Tours Barcelona
6,Runner Bean Tours Barcelona
7,Sandemans New Europe - Barcelona
8,Barcelona Segway Tour
9,Food Lover Tour


In [56]:
#Tours
tours = driver.find_element(By.XPATH, '//*[@id="search-filters"]/ul/li[6]/a')
tours.click()
time.sleep(5)

tours_page = driver.page_source
time.sleep(5)
soup4 = BeautifulSoup(tours_page, 'lxml')
tours = soup4.find_all('div', attrs={'class':'result-title'})

tour_prices_list = []
tour_ratings_list = []
tour_titles_list = []

for tour in tours:
    tour_titles_list.append(tour.text)
    links = re.findall('/AttractionProductReview.*.html', str(tour))
    for link in links:
        url_item='https://www.tripadvisor.com/' + link
        driver.get(url_item)
        tour_page = driver.page_source
        soup = BeautifulSoup(tour_page, 'lxml')
        tour_prices = soup.find_all('div', attrs={'class':'biGQs _P fiohW eIegw'})
        for tour_price in tour_prices:
            tour_prices_list.append(tour_price.text)
        
        tour_ratings = soup.find_all('div', attrs={'class':'biGQs _P fiohW hzzSG uuBRH'})
        for tour_rating in tour_ratings:
            tour_ratings_list.append(tour_rating.text)
time.sleep(3)

In [57]:
print(len(tour_prices_list))
print(len(tour_titles_list))
print(len(tour_ratings_list))

30
30
30


In [58]:
tour_titles_list

['Barcelona Half-Day Bike Tour',
 'Montserrat Half-Day Tour with Tapas and Gourmet Wines',
 'Montserrat Tour with Lunch and Gourmet Wine Tasting',
 'Half Day Barcelona Tour by Sidecar Motorcycle',
 'Girona and Costa Brava Small-Group Tour with Hotel Pickup from Barcelona',
 'Barcelona E-Bike Photography Tour',
 'Skip the Line: Park Guell and La Sagrada Familia Guided Tour',
 'Barcelona Tapas and Wine Experience Small-Group Walking Tour',
 'Kayak and Snorkel day tour to la Costa Brava from Barcelona',
 'Barcelona in One Day: Sagrada Familia, Park Guell & Old Town with Hotel Pick-up',
 'Montserrat with Cogwheel train, Tasting in a Boutique Winery with Tapas or Lunch',
 'Barcelona City Tour Hop-On Hop-Off with Optional Catamaran',
 'Barcelona Street Art Graffitti Bicycle Tour',
 'The Most Complete Full Day Private Tour with Sagrada Familia and Park Guell',
 'The Legendary Steel Donkey Off the Beaten Track Bike Tour',
 'Barcelona Tapas, Taverns and Gothic Quarter History Tour',
 'Paella Ex

In [59]:
recommended_tours = pd.DataFrame({'Tour': tour_titles_list,'Price': tour_prices_list,'Review': tour_ratings_list})
recommended_tours = recommended_tours.sort_values('Price', ascending=False).reset_index(drop=True)
recommended_tours

,Tour,Price,Review
0,Barcelona Tapas and Wine Experience Small-Grou...,$98.38,4.5
1,"Montserrat with Cogwheel train, Tasting in a B...",$98.32,5.0
2,Montserrat Small Group or Private Tour Hotel p...,$88.02,5.0
3,Kayak and Snorkel day tour to la Costa Brava f...,$82.84,5.0
4,Montserrat Half-Day Tour with Tapas and Gourme...,$82.79,5.0
5,Barcelona Highlights Private Guided Tour with ...,$714.52,5.0
6,Barcelona E-Bike Photography Tour,$68.50,5.0
7,Barcelona Electric Bike Small Group Tour with ...,$61.10,5.0
8,Fast Track Sagrada Familia Guided Tour,$52.81,4.5
9,Barcelona Segway Live-Guided Tour,$50.74,5.0


### Eventbrite.com for free events

In [60]:
#Initiating the driver and loading the website
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.eventbrite.com/')
time.sleep(2)

#accepting cookies and click on the search-box
cookies = driver.find_element(By.CLASS_NAME, 'evidon-banner-acceptbutton').click()
time.sleep(2)
search_event = driver.find_element(By.CLASS_NAME, 'eds-field-styled__input-container').click()



In [62]:
#input of the location to search 
time.sleep(5)
search_location = driver.find_element(By.ID, 'location-autocomplete')
time.sleep(2)
search_location.send_keys(place)
time.sleep(2)
search_location_list = driver.find_element(By.ID, 'location-autocomplete-listbox')
time.sleep(3)
search_location.send_keys(Keys.DOWN)
time.sleep(2)
click_location = driver.find_element(By.CLASS_NAME, 'eds-text-list-item__content-primary').click()
time.sleep(2)
search_event_click = driver.find_element(By.CLASS_NAME, 'arrow-right-chunky_svg__eds-icon--arrow-right-chunky_base').click()

In [63]:
#Search for the FREE events in Price button
time.sleep(4)
price_btn = driver.find_element(By.CSS_SELECTOR, "button[data-testid^='filter-section-button-price']")
price_btn.click()
time.sleep(4)
free_btn = driver.find_element(By.CSS_SELECTOR, "button[data-testid^='filter-section-button-free']")
free_btn.click()
time.sleep(2)
url = driver.current_url.replace('page=1', 'page=')
url

'https://www.eventbrite.com/d/spain--barcelona/free--events/?page='

In [64]:
#looping through events in the first page and creating the dataframe
time.sleep(5)
response = requests.get(url)
soup = BeautifulSoup(response.content)
events = soup.find_all('article', attrs={'class':'eds-l-pad-all-4 eds-event-card-content eds-event-card-content--list eds-event-card-content--mini eds-event-card-content--square'})
event_name = []
event_loc = []
event_date = []

for event in events: 
    name = event.find('h3', attrs={'class':'eds-event-card-content__title eds-text-color--ui-800 eds-text-bl eds-text-weight--heavy'}).text
    event_name.append(name)
    location = event.find('div', attrs={'class':'card-text--truncated__one'}).text
    event_loc.append(location)
    date = event.find('div', attrs={'class':'eds-event-card-content__sub-title eds-text-color--primary-brand eds-l-pad-bot-1 eds-l-pad-top-2 eds-text-weight--heavy eds-text-bm'}).text
    event_date.append(date)
        
events_df = pd.DataFrame({'Name':event_name,'location':event_loc,'date':event_date})

In [65]:
events_df

,Name,location,date
0,Mercado de Ropa Vintage al peso - BarcelonaMer...,VALID WORLD HALL • Barcelona,"Fri, Nov 25, 1:00 PM"
1,Tech Spirit BarcelonaTech Spirit Barcelona,Casa Llotja de Mar • Barcelona,"Wed, Nov 30, 9:00 AM"
2,Barcelona Winter Yoga Festival 2022Barcelona W...,"Barcelona • Barcelona, Spain","Sat, Nov 19, 7:00 AM"
3,"Passat, present i futur de la recerca en Alzhe...",Auditori Forum • Barcelona,"Mon, Nov 14, 7:00 PM"
4,WomanLiderTIC 2022 - Jornadas Internacionales ...,Barcelona Activa - Sede Central • Barcelona,
5,The Barcelona Startup Conference 2023The Barce...,Barcelona • Barcelona,"Tue, Apr 25, 6:00 PM"
6,El Flea. Mercadillo de segunda manoEl Flea. Me...,"Carrer de Peracamps, 11 • Barcelona","Sun, Nov 13, 10:00 AM + 1 more event"
7,Nomad Club @ MIU BarcelonaNomad Club @ MIU Bar...,Miu Japones • Barcelona,Tomorrow at 11:00 PM + 2 more events
8,BCN FinTech Afterwork #004: DeFi vs FinTechBCN...,OneCoWork Plaça Catalunya | Barcelona Coworkin...,"Wed, Nov 23, 6:30 PM"
9,HRinTech: Remote and Hybrid WorkHRinTech: Remo...,Aticco Bogatell • Barcelona,"Thu, Nov 24, 6:30 PM"


In [66]:
list_df = [main_flights_info, restaurant_recommendations_df, activities, recommended_tours, events_df]
names = ['Suggested Flights', 'Recommended Restaurants', 'To-Dos', 'Recommended Tours', 'Events']

writer = pd.ExcelWriter('travel_app.xlsx',engine='xlsxwriter')
for i,list_df in enumerate(list_df):
    list_df.to_excel(writer, sheet_name=names[i])
writer.save()


